In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import cv2
from PIL import Image
import numpy as np
import random
import tqdm
import json
import matplotlib.pyplot as plt

from visual_tokenizer import get_visual_tokenizer
from utils.visualization import visualize_masks

In [ ]:
image_dir = '/home/dchenbs/workspace/datasets/coco2017/images/val2017'

# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_01_11580'
# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_02_11598'
# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_03_10049'
# image_dir = '/home/dchenbs/workspace/datasets/CLEVR_v1.0/images/test'
# image_dir =  '/home/dchenbs/workspace/datasets/sa1b/sa_000001'
# image_dir =  '/home/dchenbs/workspace/subobjects-dev/assets/examples'

# walk through the image_dir and get all the image paths
image_paths = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
            image_paths.append(os.path.join(root, file))

print(len(image_paths))

In [ ]:
image_resolution = 1024
max_tokens = 256

# config = json.load(open('configs/visual_tokenizer/patch_8_per_side_random.json'))
config = json.load(open('configs/visual_tokenizer/patch_8_per_side_raster.json'))
# config = json.load(open('configs/visual_tokenizer/directsam_0424.json'))
# config = json.load(open('configs/visual_tokenizer/directsam_tiny.json'))

visual_tokenizer = get_visual_tokenizer(**config, image_resolution=image_resolution, max_tokens=max_tokens)

In [ ]:
for _ in range(3):
    image_path = random.choice(image_paths)
    image = Image.open(image_path).resize((image_resolution, image_resolution))
    batch_masks = visual_tokenizer(image)

    plt.figure(figsize=(20, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.imshow(visualize_masks(image, batch_masks[0]))
    plt.title((np.sum(batch_masks[0], axis=(1, 2)) > 0).sum())


    labels = np.zeros_like(batch_masks[0][0]).astype(np.int32)
    for i, mask in enumerate(batch_masks[0]):
        if np.sum(mask) == 0:
            continue
        labels += (i + 1) * mask

    plt.subplot(1, 3, 3)
    plt.imshow(labels, cmap='plasma')
    plt.title('order')


    plt.show()